<b>We can use 4 common way to handle with imbalancing data.<br>
Handling with imbalanced data is really important for machine learning models.

- UnderSampling
- OverSampling (blind copy)
- OverSampling (SMOTE)
- Ensemble

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [47]:
df = pd.read_csv('Datasets/customer_churn.csv')

In [6]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [15]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [16]:
for i in list(df.select_dtypes(object)):
    print("{} => {}".format(i,df[i].unique()))

customerID => ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
gender => ['Female' 'Male']
Partner => ['Yes' 'No']
Dependents => ['No' 'Yes']
PhoneService => ['No' 'Yes']
MultipleLines => ['No phone service' 'No' 'Yes']
InternetService => ['DSL' 'Fiber optic' 'No']
OnlineSecurity => ['No' 'Yes' 'No internet service']
OnlineBackup => ['Yes' 'No' 'No internet service']
DeviceProtection => ['No' 'Yes' 'No internet service']
TechSupport => ['No' 'Yes' 'No internet service']
StreamingTV => ['No' 'Yes' 'No internet service']
StreamingMovies => ['No' 'Yes' 'No internet service']
Contract => ['Month-to-month' 'One year' 'Two year']
PaperlessBilling => ['Yes' 'No']
PaymentMethod => ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
TotalCharges => ['29.85' '1889.5' '108.15' ... '346.45' '306.6' '6844.5']
Churn => ['No' 'Yes']


In [17]:
labels1 = ['No','Yes','No internet service']
labels2 = ['No','Yes']
labels3 = ['No phone service', 'No', 'Yes']
labels4 = ['No','DSL','Fiber optic']
labels5 = ['Female','Male']

In [48]:
for i in ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']:
    df[i] = df[i].apply(lambda x: labels1.index(x))

In [57]:
df.dtypes[df.dtypes==object]

TotalCharges    object
dtype: object

In [50]:
df.drop('customerID',axis=1,inplace=True)

In [51]:
for i in ['Dependents','Partner','PhoneService','PaperlessBilling','Churn']:
    df[i] = df[i].apply(lambda x: labels2.index(x))

In [52]:
df.MultipleLines = df.MultipleLines.apply(lambda x: labels3.index(x))

In [53]:
df.InternetService = df.InternetService.apply(lambda x: labels4.index(x))

In [54]:
df.gender = df.gender.apply(lambda x: labels5.index(x))

In [55]:
df = pd.get_dummies(data = df,columns=['Contract','PaymentMethod'])

In [62]:
aykirilar = df.TotalCharges.isin([" "])[df.TotalCharges.isin([" "])==True]

In [67]:
df.drop(aykirilar.index,inplace=True)

In [73]:
df.TotalCharges=df.TotalCharges.astype(float)

In [75]:
df.dtypes

gender                                       int64
SeniorCitizen                                int64
Partner                                      int64
Dependents                                   int64
tenure                                       int64
PhoneService                                 int64
MultipleLines                                int64
InternetService                              int64
OnlineSecurity                               int64
OnlineBackup                                 int64
DeviceProtection                             int64
TechSupport                                  int64
StreamingTV                                  int64
StreamingMovies                              int64
PaperlessBilling                             int64
MonthlyCharges                             float64
TotalCharges                               float64
Churn                                        int64
Contract_Month-to-month                      uint8
Contract_One year              

<b>Everything seems good lets see 

# Without Balancing

In [77]:
X = df.drop('Churn',axis=1)
y = df.Churn

In [78]:
from sklearn.preprocessing import MinMaxScaler

column_names = X.columns
scaler = MinMaxScaler().fit(X)
X = pd.DataFrame(scaler.transform(X),columns=column_names)

In [82]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [90]:
X.shape

(7032, 24)

In [133]:
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1,activation='sigmoid')
])

In [134]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [135]:
model.fit(x_train,y_train,epochs=50)

Epoch 1/50
154/154 [==============================] - 1s 1ms/step - loss: 0.5204 - accuracy: 0.7381
Epoch 2/50
154/154 [==============================] - 0s 1ms/step - loss: 0.4501 - accuracy: 0.7779
Epoch 3/50
154/154 [==============================] - 0s 1ms/step - loss: 0.4415 - accuracy: 0.7855
Epoch 4/50
154/154 [==============================] - 0s 1ms/step - loss: 0.4371 - accuracy: 0.7936
Epoch 5/50
154/154 [==============================] - 0s 1ms/step - loss: 0.4304 - accuracy: 0.7918
Epoch 6/50
154/154 [==============================] - 0s 1ms/step - loss: 0.4274 - accuracy: 0.7942
Epoch 7/50
154/154 [==============================] - 0s 1ms/step - loss: 0.4282 - accuracy: 0.7976
Epoch 8/50
154/154 [==============================] - 0s 1ms/step - loss: 0.4222 - accuracy: 0.7968
Epoch 9/50
154/154 [==============================] - 0s 1ms/step - loss: 0.4202 - accuracy: 0.8001
Epoch 10/50
154/154 [==============================] - 0s 1ms/step - loss: 0.4188 - accuracy: 0.7993

In [136]:
model.evaluate(x_test,y_test)

66/66 [==============================] - 0s 1ms/step - loss: 0.4391 - accuracy: 0.7782


[0.4391484558582306, 0.7781990766525269]

In [138]:
prediction = model.predict(x_test)
prediction_step = list()
for i in prediction:
    prediction_step.append(round(i[0]))

66/66 [==============================] - 0s 836us/step


In [141]:
print(classification_report(prediction_step,y_test))

              precision    recall  f1-score   support

           0       0.86      0.84      0.85      1597
           1       0.54      0.59      0.56       513

    accuracy                           0.78      2110
   macro avg       0.70      0.71      0.71      2110
weighted avg       0.79      0.78      0.78      2110



<h1 style="color:red"> With Balancing

In [142]:
df.Churn.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

<h2 style="color:purple"> Under sampling

In [143]:
churn_0s = df[df.Churn==0]
churn_1s = df[df.Churn==1]

In [147]:
under_sampling_churn0s = churn_0s.sample(1869)

In [154]:
print(under_sampling_churn0s.count()['gender'])
print(churn_1s.count()['gender'])

1869
1869


<b style="color:blue">
Now we have the same value count of 1 and 0 but we ignored and removed so many data and all data might be very important , so this technich might be cause big information lose

<h2 style="color:purple"> Oversampling (blind copy)

In [156]:
churn_0s = df[df.Churn==0]
churn_1s = df[df.Churn==1]

In [158]:
churn_0s.count().gender

5163

In [160]:
churn_1s = churn_1s.sample(5163,replace=True)

In [163]:
print(churn_0s.count()['gender'])
print(churn_1s.count()['gender'])

5163
5163


<b style="color:blue">Now we have the same value count of 1 and 0 but we copied the same samples this can be better then under sampling but not very efficient

<h2 style="color:purple"> Oversampling (SMOTE)

In [166]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='auto',random_state=42)
x_sm,y_sm = smote.fit_resample(X,y)

In [167]:
y_sm.value_counts()

0    5163
1    5163
Name: Churn, dtype: int64

<b style="color:blue">Now we have again same count of different values. And with Knn value we produced synonim samples. This is generally the most efficient way to avoid imbalancing in datasets.

### Creating model again with balansed dataset

In [169]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_sm,y_sm,test_size=0.3,random_state=42)

In [170]:
x_train.shape

(7228, 24)

In [171]:
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1,activation='sigmoid')
])

In [172]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [173]:
model.fit(x_train,y_train,epochs=50)

Epoch 1/50
226/226 [==============================] - 1s 1ms/step - loss: 0.5578 - accuracy: 0.7201
Epoch 2/50
226/226 [==============================] - 0s 1ms/step - loss: 0.5041 - accuracy: 0.7561
Epoch 3/50
226/226 [==============================] - 0s 1ms/step - loss: 0.4887 - accuracy: 0.7591
Epoch 4/50
226/226 [==============================] - 0s 1ms/step - loss: 0.4850 - accuracy: 0.7637
Epoch 5/50
226/226 [==============================] - 0s 1ms/step - loss: 0.4801 - accuracy: 0.7662
Epoch 6/50
226/226 [==============================] - 0s 1ms/step - loss: 0.4749 - accuracy: 0.7674
Epoch 7/50
226/226 [==============================] - 0s 1ms/step - loss: 0.4700 - accuracy: 0.7684
Epoch 8/50
226/226 [==============================] - 0s 1ms/step - loss: 0.4660 - accuracy: 0.7741
Epoch 9/50
226/226 [==============================] - 0s 1ms/step - loss: 0.4651 - accuracy: 0.7731
Epoch 10/50
226/226 [==============================] - 0s 2ms/step - loss: 0.4619 - accuracy: 0.7775

In [174]:
model.evaluate(x_test,y_test)

97/97 [==============================] - 0s 1ms/step - loss: 0.4534 - accuracy: 0.7811


[0.45336511731147766, 0.78114914894104]

In [175]:
prediction = model.predict(x_test)
prediction_step = list()
for i in prediction:
    prediction_step.append(round(i[0]))

97/97 [==============================] - 0s 872us/step


In [176]:
print(classification_report(prediction_step,y_test))

              precision    recall  f1-score   support

           0       0.73      0.82      0.77      1397
           1       0.83      0.75      0.79      1701

    accuracy                           0.78      3098
   macro avg       0.78      0.78      0.78      3098
weighted avg       0.79      0.78      0.78      3098



# Now we have definitely better classification report than imbalanced